In [1]:
import sys
import numpy as np
# setting path
sys.path.append('../')


In [2]:
dataset_path = "../datasets/movies/ratings_small.csv"
train_path = "../datasets/movies/ratings_small_train.csv"
test_path = "../datasets/movies/ratings_small_test.csv"

In [30]:
from split_data import split_data

split_data(dataset_path, 0.995)

In [31]:
# Load csv
train_data = np.loadtxt(train_path, delimiter=",", skiprows=0, dtype="i4,i4,f,i4")
test_data = np.loadtxt(test_path, delimiter=",", skiprows=0, dtype="i4,i4,f,i4")
train_data[:6]

array([(285,  519, 2. ,  965089184), ( 34, 1527, 3. ,  973747685),
       (620, 7254, 3.5, 1455532188), (239, 3120, 3. ,  991862309),
       (461,  333, 4. , 1090907428), (213, 1422, 4.5, 1462635638)],
      dtype=[('f0', '<i4'), ('f1', '<i4'), ('f2', '<f4'), ('f3', '<i4')])

In [32]:
from sparse_array import NDSparseArray
from datetime import datetime

def sparse_data(data):
    # Put timestamps into "bins" of year and month only
    timestamps = [line[3] for line in data]
    timestamps = [datetime.fromtimestamp(timestamp) for timestamp in timestamps]
    mini = min(timestamps).year
    timestamps = [(timestamp.year-mini)*12+timestamp.month for timestamp in timestamps]

    # Load user Ids and movieIds
    userIds = [line[0] for line in data]
    movieIds = [line[1] for line in data]
    ratings = [line[2] for line in data]

    # Create sparse array
    Y = NDSparseArray((max(userIds)+1, max(movieIds)+1, max(timestamps)+1))

    for i in range(len(userIds)):
        Y[userIds[i], movieIds[i], timestamps[i]] = ratings[i]

    return Y

Y = sparse_data(train_data)
Y_test = sparse_data(test_data)
Y[564, 1831, 71]  # 71 = (2000-1995) * 12 + 11

1.0

In [33]:
from tensor_factorization import tensor_factorization, initialize_factorization, D, factorization_cycle, Lambda

U, M, C, S = initialize_factorization(Y, D(27, 76, 13), Lambda(0.1, 0.1, 0.1, 0.1))

In [34]:
from tensor_factorization import Lambda

la = Lambda(0.000001, 0.0000001, 0.000001, 0.000001)
for t in range(30, 33):
    def coef(s):
        return 0.01 * 1 / (t ** 0.5)
    U, M, C, S = factorization_cycle(U, M, C, S, Y, coef, la)

1.0308727480147315 99503/99503     
0.7013795892017465 99503/99503     
0.6867116854105575 99503/99503     


In [29]:
from tensor_factorization import evaluate

SE = 0
n = len(Y_test.elements)
for i, j, k in Y_test.indexes():
    rating = Y_test[i, j, k]
    evalRating = evaluate(U, M, C, S, i, j, k)
    error = abs(rating - evalRating)
    SE += error
MAE = SE / n
print(MAE)

1.016322425879298
